In [1]:
import pandas as pd
import numpy as np
import janitor as jntr
import difflib

characteristic_data = pd.read_csv("hego_data/hego_17_18_salary_by_characteristic.csv", skiprows = 13).clean_names()
institution_data = pd.read_csv("hego_data/hego_17_18_salary_by_institution.csv", skiprows = 13).clean_names()
subject_data = pd.read_csv("hego_data/hego_17_18_salary_by_subject.csv", skiprows = 13).clean_names()
uni_codes = pd.read_excel("hego_data/institution-and-campus-codes-2018-entry.xls").clean_names()
rankings_data = pd.read_csv("hego_data/Complete_University_Guide_University_League_Table.csv", skiprows=4).clean_names()

In [30]:
pd.set_option('display.max_rows', 500)

To begin with and to make sure everything is working, only one year will be used. All the other data that has been read in refers only to this year (17/18 that is) so only the ranking dataset needs filtered.

In [2]:
rankings_2017 = rankings_data.loc[rankings_data["year"] == 2017].copy()

I've been having problems with these datasets whilst trying to join them on the instituion/provider names. I've tried to do this by using what's known as fuzzy merging. This has worked to a degree, but has thrown up some significant issues. I'm now going to try and create a more concise merging process so that I can see if the merging and then correcting I've used has worked.

In [3]:
institution_data_for_merge = institution_data.copy()

In [4]:
just_2017_rankings = rankings_2017.loc[:,["rank", "institution"]].copy()

Now to define our fuzzy merge function using the following source: 

https://stackoverflow.com/questions/13636848/is-it-possible-to-do-fuzzy-match-merge-with-python-pandas/60908516#60908516

In [5]:
def fuzzy_merge(df1, df2, left_on, right_on, how='inner', cutoff=0.6):
    df_other= df2.copy()
    df_other[left_on] = [get_closest_match(x, df1[left_on], cutoff) 
                         for x in df_other[right_on]]
    return df1.merge(df_other, on=left_on, how=how)

def get_closest_match(x, other, cutoff):
    matches = difflib.get_close_matches(x, other, cutoff=cutoff)
    return matches[0] if matches else None

Through trial and error, I've worked out that the function was getting a little confused when it came to certain titles, especially "The University of....". With this in mind, I altered the provider names before the merge.

In [6]:
institution_data_for_merge["provider_name"] = institution_data_for_merge["provider_name"].str.replace("University", '')

Now time for the merge. This will require further modification and for this reason is being labelled provisional:

In [7]:
provisional_merge = fuzzy_merge(institution_data_for_merge, just_2017_rankings, left_on='provider_name', right_on='institution').copy()

So just like last time, there's a certain amount of institutions that have been misclassified and need to be hard-coded out and in:

In [8]:
provisional_merge = provisional_merge[~provisional_merge['provider_name'].isin(['College London', 'of Cumbria', 'Arden', 'Bath Spa', 'Birmingham City', 'Liverpool Hope', 'London School of Theology', 'Southport College'])]

In [9]:
second_institution_merge_data = institution_data.copy()

In [10]:
second_institution_merge_data['provider_name'] = second_institution_merge_data['provider_name'].replace({'Imperial College of Science, Technology and Medicine': 'Imperial College London'})

In [11]:
second_institution_merge_data = second_institution_merge_data[second_institution_merge_data["provider_name"].isin(['Imperial College London', 'University College London', 'University of Northumbria at Newcastle', 'University of Cumbria', 'Arden University', 'The University of Aberdeen', 'The University of Bath', 'Bath Spa University', 'The University of Liverpool', 'Liverpool Hope University', 'London School of Economics and Political Science', 'The University of Southampton', 'Solent University', 'The University of Birmingham', 'The University of York', 'The University of Kent', 'Royal Holloway and Bedford New College', 'The University of Essex', 'Goldsmiths College', 'The University of Hull', 'University of St Mark and St John'])]


In [12]:
conditions = [
    (second_institution_merge_data['provider_name'] == "Imperial College London"),
    (second_institution_merge_data['provider_name'] == "University College London"),
    (second_institution_merge_data['provider_name'] == "University of Northumbria at Newcastle"),
    (second_institution_merge_data['provider_name'] == "University of Cumbria"),
    (second_institution_merge_data['provider_name'] == "Arden University"),
    (second_institution_merge_data['provider_name'] == "The University of Aberdeen"),
    (second_institution_merge_data['provider_name'] == "The University of Bath"),
    (second_institution_merge_data['provider_name'] == "Bath Spa University"),
    (second_institution_merge_data['provider_name'] == "The University of Liverpool"),
    (second_institution_merge_data['provider_name'] == "Liverpool Hope University"),
    (second_institution_merge_data['provider_name'] == "London School of Economics and Political Science"),
    (second_institution_merge_data['provider_name'] == "The University of Southampton"),
    (second_institution_merge_data['provider_name'] == "Solent University"),
    (second_institution_merge_data['provider_name'] == "The University of Birmingham"),
    (second_institution_merge_data['provider_name'] == "The University of York"),
    (second_institution_merge_data['provider_name'] == "The University of Kent"),
    (second_institution_merge_data['provider_name'] == "Royal Holloway and Bedford New College"),
    (second_institution_merge_data['provider_name'] == "The University of Essex"),
    (second_institution_merge_data['provider_name'] == "Goldsmiths College"),
    (second_institution_merge_data['provider_name'] == "The University of Hull"),
    (second_institution_merge_data['provider_name'] == "University of St Mark and St John")
]

values = [4, 10, 59, 121,"NA",42,11,76,38,83,3,17,115,15, 20, 23, 37, 41, 51, 68, 124]

second_institution_merge_data['rank'] = np.select(conditions, values)

In [13]:
second_institution_merge_data["institution"] = second_institution_merge_data["provider_name"]

In [14]:
all_institutions_out_n_ranks_2017 = pd.concat([provisional_merge, second_institution_merge_data])

That should be the main body done, although there's still a few little extra mistakes to correct inside there.

In [16]:
all_inst_minus_mistakes = all_institutions_out_n_ranks_2017.loc[~((all_institutions_out_n_ranks_2017["provider_name"] == " College London") & (all_institutions_out_n_ranks_2017["institution"] == "Imperial College London")),:]
all_inst_minus_mistakes = all_inst_minus_mistakes.loc[~((all_inst_minus_mistakes["provider_name"] == "Arden ") & (all_inst_minus_mistakes["institution"] == "Aberdeen")),:]
all_inst_minus_mistakes = all_inst_minus_mistakes.loc[~((all_inst_minus_mistakes["provider_name"] == "Arden University") & (all_inst_minus_mistakes["institution"] == "Arden University")),:]
all_inst_minus_mistakes = all_inst_minus_mistakes.loc[~((all_inst_minus_mistakes["provider_name"] == "Bath Spa ") & (all_inst_minus_mistakes["institution"] == "Bath")),:]
all_inst_minus_mistakes = all_inst_minus_mistakes.loc[~((all_inst_minus_mistakes["provider_name"] == "Birmingham City ") & (all_inst_minus_mistakes["institution"] == "Birmingham")),:]
all_inst_minus_mistakes = all_inst_minus_mistakes.loc[~((all_inst_minus_mistakes["provider_name"] == "Liverpool Hope ") & (all_inst_minus_mistakes["institution"] == "Liverpool")),:]
                                                                   

In [17]:
all_institutions_out_n_ranks_2017 = all_inst_minus_mistakes.copy()

In [18]:
all_institutions_out_n_ranks_2017["rank"] = pd.to_numeric(all_institutions_out_n_ranks_2017["rank"])

In [19]:
all_institutions_out_n_ranks_2017 = all_institutions_out_n_ranks_2017.loc[~(all_institutions_out_n_ranks_2017["institution"].isin(["Southampton", "Northumbria", "Bath Spa", "Liverpool Hope", "Cumbria"]))]

Cool, so that looks like the main institution and ranking merge is finally done. Now to get the uni codes done. 

In [20]:
uni_codes_for_merge = uni_codes.clean_names().copy()

In [21]:
uni_codes_for_merge = uni_codes_for_merge.loc[:,["name_abbreviation", "official_name", "name"]].copy()

In [24]:
uni_codes_for_merge = uni_codes_for_merge.drop_duplicates()

In [25]:
allinst_copy_for_code_merge = all_institutions_out_n_ranks_2017.copy()

Now to try the second merge, with the codes:

In [32]:
code_merge_test = fuzzy_merge(allinst_copy_for_code_merge, uni_codes_for_merge, left_on = "institution", right_on = 'name').copy()

In [33]:
code_merge_test

,ukprn,provider_name,country_of_provider,mode_of_former_study,skill_group,work_population_marker,salary_band,number,rank,institution,name_abbreviation,official_name,name
0,10000291.0,Anglia Ruskin,England,Full-time,High skilled,Paid employment is an activity,"Less than £15,000",5,110,Anglia Ruskin,ARU,Anglia Ruskin University,Anglia Ruskin University
1,10000291.0,Anglia Ruskin,England,Full-time,High skilled,Paid employment is an activity,"£15,000 - £17,999",40,110,Anglia Ruskin,ARU,Anglia Ruskin University,Anglia Ruskin University
2,10000291.0,Anglia Ruskin,England,Full-time,High skilled,Paid employment is an activity,"£18,000 - £20,999",80,110,Anglia Ruskin,ARU,Anglia Ruskin University,Anglia Ruskin University
3,10000291.0,Anglia Ruskin,England,Full-time,High skilled,Paid employment is an activity,"£21,000 - £23,999",140,110,Anglia Ruskin,ARU,Anglia Ruskin University,Anglia Ruskin University
4,10000291.0,Anglia Ruskin,England,Full-time,High skilled,Paid employment is an activity,"£24,000 - £26,999",265,110,Anglia Ruskin,ARU,Anglia Ruskin University,Anglia Ruskin University
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85295,10007167.0,The University of York,England,Part-time,All,Paid employment is most important activity,"£39,000+",5,20,The University of York,SALF,The University of Salford,The University of Salford
85296,10007167.0,The University of York,England,Part-time,All,Paid employment is most important activity,"£39,000+",5,20,The University of York,UCS,University of Suffolk,University of Suffolk
85297,10007167.0,The University of York,England,Part-time,All,Paid employment is most important activity,"£39,000+",5,20,The University of York,WARW,The University of Warwick,The University of Warwick
85298,10007167.0,The University of York,England,Part-time,All,Paid employment is most important activity,"£39,000+",5,20,The University of York,WORCS,University of Worcester,University of Worcester


In [34]:
code_merge_groups = code_merge_test.groupby(['institution', 'name_abbreviation']).size().reset_index().rename(columns={0:'count'})

In [35]:
code_merge_groups

,institution,name_abbreviation,count
0,Anglia Ruskin,ARU,480
1,Arts University Bournemouth,AUCB,400
2,Arts University Bournemouth,PORT,400
3,Bath Spa University,BANGR,400
4,Bath Spa University,BASPA,400
5,Bath Spa University,BPP,400
6,Bath Spa University,CARDF,400
7,Bath Spa University,DUR,400
8,Bath Spa University,FAL,400
9,Bath Spa University,LANCR,400


Ok, so this is where we ran into problems on the first atttempt, although one of the big mistakes before was using one of the original datasets rather than our edited version (institution_outcomes_rankings_2017 vs all_institutions_out_n_ranks_2017). Time to have another go:

In [36]:
minus_college_codes_for_merge = uni_codes_for_merge.replace({'name': {'UCL (University College London': 'UCL', 'Imperial College London': 'ICL', "King's College London (University of London)": 'KCL'}})

In [37]:
minus_college_codes_for_merge_final = minus_college_codes_for_merge.loc[~minus_college_codes_for_merge["name"].str.contains("College", case = False)].copy()

In [39]:
minus_college_codes_for_merge_final = minus_college_codes_for_merge_final.replace({'name': {'UCL':'UCL (University College London' , 'ICL': 'Imperial College London' ,'KCL': "King's College London (University of London)"}})


In [45]:
second_code_merge_test = fuzzy_merge(minus_college_codes_for_merge_final, allinst_copy_for_code_merge, left_on = 'name', right_on = "institution", cutoff=0.4).copy()

In [54]:
second_code_merge_groups = second_code_merge_test.groupby(['institution', 'provider_name', 'name', 'name_abbreviation', 'rank']).size().reset_index().rename(columns={0:'count'})

In [55]:
second_code_merge_groups

,institution,provider_name,name,name_abbreviation,rank,count
0,Abertay,Abertay,Abertay University,ABTAY,86,440
1,Aberystwyth,Aberystwyth,Aberystwyth University,ABWTH,87,440
2,Anglia Ruskin,Anglia Ruskin,Anglia Ruskin University,ARU,110,480
3,Arts University Bournemouth,The Arts Bournemouth,Arts University Bournemouth,AUCB,76,400
4,Aston,Aston,Access to Music,ACCM,30,480
5,Bangor,Bangor,Bangor University,BANGR,62,440
6,Bath Spa University,Bath Spa University,Bath Spa University,BASPA,76,400
7,Bedfordshire,of Bedfordshire,University of Bedfordshire,BEDS,120,480
8,Birmingham City,Birmingham City,Birmingham City University,BCITY,95,440
9,Bishop Grosseteste,Bishop Grosseteste,Bishop Grosseteste University,BGU,119,320


In [56]:
second_code_merge_groups.sort_values("rank", ascending = True)

,institution,provider_name,name,name_abbreviation,rank,count
18,Cambridge,The of Cambridge,University of Cambridge,CAM,1,320
71,Oxford,The of Oxford,Oxford University,OXF,2,400
60,London School of Economics and Political Science,London School of Economics and Political Science,London School of Economics and Political Scien...,LSE,3,320
47,Imperial College London,Imperial College London,Imperial College London,IMP,4,320
89,St Andrews,The of St Andrews,University of St Andrews,STA,5,440
30,Durham,of Durham,Durham University,DUR,6,400
62,Loughborough,Loughborough,Loughborough University,LBRO,7,440
118,Warwick,The of Warwick,The University of Warwick,WARW,8,440
51,Lancaster,The of Lancaster,Lancaster University,LANCR,9,320
110,University College London,College London,SOAS University of London,SOAS,10,400


In [57]:
institution_group_ordered = all_institutions_out_n_ranks_2017.groupby(['institution', 'provider_name', 'rank']).size().reset_index().rename(columns={0:'count'}).copy()

In [59]:
institution_group_ordered.sort_values("rank", ascending = True)

,institution,provider_name,rank,count
18,Cambridge,The of Cambridge,1,320
72,Oxford,The of Oxford,2,400
60,London School of Economics and Political Science,London School of Economics and Political Science,3,320
47,Imperial College London,Imperial College London,4,320
90,St Andrews,The of St Andrews,5,440
30,Durham,of Durham,6,400
62,Loughborough,Loughborough,7,440
119,Warwick,The of Warwick,8,440
51,Lancaster,The of Lancaster,9,320
111,University College London,College London,10,400


In [ ]:
second_code_merge_groups = second_code_merge_test.groupby(['institution', 'provider_name', 'name', 'name_abbreviation', 'rank']).size().reset_index().rename(columns={0:'count'})

In [49]:
all_institutions_out_n_ranks_2017.describe(include = "all")

,ukprn,provider_name,country_of_provider,mode_of_former_study,skill_group,work_population_marker,salary_band,number,rank,institution
count,5.574000e+04,55740,55740,55740,55740,55740,55740,55740.000000,55740.000000,55740
unique,NaN,128,5,3,4,2,10,NaN,NaN,127
top,NaN,The of Chichester,All,All,All,Paid employment is an activity,"£15,000 - £17,999",NaN,NaN,University College London
freq,NaN,480,27870,20480,14720,27900,5574,NaN,NaN,800
mean,1.000675e+07,NaN,NaN,NaN,NaN,NaN,NaN,26.819699,65.428776,NaN
std,4.547923e+03,NaN,NaN,NaN,NaN,NaN,NaN,52.790819,36.424347,NaN
min,1.000029e+07,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000,NaN
25%,1.000534e+07,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,34.000000,NaN
50%,1.000716e+07,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,67.000000,NaN
75%,1.000779e+07,NaN,NaN,NaN,NaN,NaN,NaN,25.000000,97.000000,NaN


In [60]:
min_prov_all_institutions_out_n_ranks_2017 = all_institutions_out_n_ranks_2017.drop('provider_name', 1).copy()

In [97]:
min_prov_all_institutions_out_n_ranks_2017 = min_prov_all_institutions_out_n_ranks_2017.drop_duplicates(keep = 'first')

In [98]:
extra_institution_group_ordered = min_prov_all_institutions_out_n_ranks_2017.groupby(['institution', 'rank']).size().reset_index().rename(columns={0:'count'}).copy()

In [99]:
extra_institution_group_ordered

,institution,rank,count
0,Abertay,86,440
1,Aberystwyth,87,440
2,Anglia Ruskin,110,480
3,Arts University Bournemouth,76,400
4,Aston,30,480
5,Bangor,62,440
6,Bath Spa University,76,400
7,Bedfordshire,120,480
8,Birmingham City,95,440
9,Bishop Grosseteste,119,320


In [101]:
min_prov_all_institutions_out_n_ranks_2017.duplicated().any() 

False

In [102]:
institution_data

,ukprn,provider_name,country_of_provider,mode_of_former_study,skill_group,work_population_marker,salary_band,number
0,10008071.0,AA School of Architecture,England,Full-time,High skilled,Paid employment is an activity,"Less than £15,000",0
1,10008071.0,AA School of Architecture,England,Full-time,High skilled,Paid employment is an activity,"£15,000 - £17,999",0
2,10008071.0,AA School of Architecture,England,Full-time,High skilled,Paid employment is an activity,"£18,000 - £20,999",0
3,10008071.0,AA School of Architecture,England,Full-time,High skilled,Paid employment is an activity,"£21,000 - £23,999",0
4,10008071.0,AA School of Architecture,England,Full-time,High skilled,Paid employment is an activity,"£24,000 - £26,999",0
...,...,...,...,...,...,...,...,...
113875,NaN,Total,Wales,Part-time,Medium skilled,Paid employment is most important activity,"£27,000 - £29,999",0
113876,NaN,Total,Wales,Part-time,Medium skilled,Paid employment is most important activity,"£30,000 - £32,999",5
113877,NaN,Total,Wales,Part-time,Medium skilled,Paid employment is most important activity,"£33,000 - £35,999",10
113878,NaN,Total,Wales,Part-time,Medium skilled,Paid employment is most important activity,"£36,000 - £38,999",5


In [117]:
leeds_check = institution_data.loc[institution_data["ukprn"] == 10003854.0].copy()

In [118]:
leeds_check

,ukprn,provider_name,country_of_provider,mode_of_former_study,skill_group,work_population_marker,salary_band,number
52460,10003854.0,Leeds Arts University,England,Full-time,High skilled,Paid employment is an activity,"Less than £15,000",5
52461,10003854.0,Leeds Arts University,England,Full-time,High skilled,Paid employment is an activity,"£15,000 - £17,999",5
52462,10003854.0,Leeds Arts University,England,Full-time,High skilled,Paid employment is an activity,"£18,000 - £20,999",25
52463,10003854.0,Leeds Arts University,England,Full-time,High skilled,Paid employment is an activity,"£21,000 - £23,999",15
52464,10003854.0,Leeds Arts University,England,Full-time,High skilled,Paid employment is an activity,"£24,000 - £26,999",10
52465,10003854.0,Leeds Arts University,England,Full-time,High skilled,Paid employment is an activity,"£27,000 - £29,999",0
52466,10003854.0,Leeds Arts University,England,Full-time,High skilled,Paid employment is an activity,"£30,000 - £32,999",0
52467,10003854.0,Leeds Arts University,England,Full-time,High skilled,Paid employment is an activity,"£33,000 - £35,999",0
52468,10003854.0,Leeds Arts University,England,Full-time,High skilled,Paid employment is an activity,"£36,000 - £38,999",0
52469,10003854.0,Leeds Arts University,England,Full-time,High skilled,Paid employment is an activity,"£39,000+",0


In [111]:
min_prov_all_institutions_out_n_ranks_2017.loc[min_prov_all_institutions_out_n_ranks_2017["institution"] == "Leeds"]

,ukprn,country_of_provider,mode_of_former_study,skill_group,work_population_marker,salary_band,number,rank,institution
25540,10003854.0,England,Full-time,High skilled,Paid employment is an activity,"Less than £15,000",5,16,Leeds
25541,10003854.0,England,Full-time,High skilled,Paid employment is an activity,"£15,000 - £17,999",5,16,Leeds
25542,10003854.0,England,Full-time,High skilled,Paid employment is an activity,"£18,000 - £20,999",25,16,Leeds
25543,10003854.0,England,Full-time,High skilled,Paid employment is an activity,"£21,000 - £23,999",15,16,Leeds
25544,10003854.0,England,Full-time,High skilled,Paid employment is an activity,"£24,000 - £26,999",10,16,Leeds
25545,10003854.0,England,Full-time,High skilled,Paid employment is an activity,"£27,000 - £29,999",0,16,Leeds
25546,10003854.0,England,Full-time,High skilled,Paid employment is an activity,"£30,000 - £32,999",0,16,Leeds
25547,10003854.0,England,Full-time,High skilled,Paid employment is an activity,"£33,000 - £35,999",0,16,Leeds
25548,10003854.0,England,Full-time,High skilled,Paid employment is an activity,"£36,000 - £38,999",0,16,Leeds
25549,10003854.0,England,Full-time,High skilled,Paid employment is an activity,"£39,000+",0,16,Leeds


Looks like the Leeds Data is wrong so we'll need to take this out and put a replacement back in. Then we can finalize the removal of the UCL duplicates and then finally get around to making the final code adjustments.